# Listando todos os arquivos nos sub-diretórios

Para calcular a média de cada eletrodo em cada arquivo, é necessário recuperar o nome de cada arquivos nas sub-pastas.



In [41]:
import os

def get_sub_files(path_name, files=[]):

    subpaths = os.listdir(path_name)

    for path in subpaths:
        full_name = path_name + path
        if os.path.isdir(full_name):
            get_sub_files(full_name + '/', files) # goes to next sub-level and concat with the current list of files
        elif not path_name.startswith('.'):
            files.append(full_name)

    return files

all_files = get_sub_files('dataset/')

# Carregando os dados dos eletrodos

Com todos os arquivos listados, basta abrir cada um e salvar os valores de cada eletrodo. Se dois arquivos contêm os eletrodos, por exemplo, `P17`, os seus valores serão somados e calculados a média.

In [ ]:
def process_file(output, file):

    for line in file.readlines():
        if line.startswith('#'):
            continue

        line = line.split()
        #print(line)
        try:
            # 41 FP2 168 -4.720
            output[line[1]][int(line[2])] += float(line[3])
        except KeyError:
            output[line[1]] = list(range(256))
            output[line[1]][int(line[2])] = float(line[3])

def process_files(files):
    output = {}

    for file in files:
        print(file)
        process_file(output, open(file))

process_files(all_files)
print()

dataset/full/co2a0000382/co2a0000382.rd.077
dataset/full/co2a0000382/co2a0000382.rd.117
dataset/full/co2a0000382/co2a0000382.rd.038
dataset/full/co2a0000382/co2a0000382.rd.019
dataset/full/co2a0000382/co2a0000382.rd.067
dataset/full/co2a0000382/co2a0000382.rd.078
dataset/full/co2a0000382/co2a0000382.rd.015
dataset/full/co2a0000382/co2a0000382.rd.054
dataset/full/co2a0000382/co2a0000382.rd.111
dataset/full/co2a0000382/co2a0000382.rd.016
dataset/full/co2a0000382/co2a0000382.rd.119
dataset/full/co2a0000382/co2a0000382.rd.041
dataset/full/co2a0000382/co2a0000382.rd.112
dataset/full/co2a0000382/co2a0000382.rd.105
dataset/full/co2a0000382/co2a0000382.rd.096
dataset/full/co2a0000382/co2a0000382.rd.100
dataset/full/co2a0000382/co2a0000382.rd.003
dataset/full/co2a0000382/co2a0000382.rd.102
dataset/full/co2a0000382/co2a0000382.rd.099
dataset/full/co2a0000382/co2a0000382.rd.075
dataset/full/co2a0000382/co2a0000382.rd.107
dataset/full/co2a0000382/co2a0000382.rd.014
dataset/full/co2a0000382/co2a000

In [8]:
electrode = 'O2'
fileName = '/a_1_co2a0000364/co2a0000364.rd.000'

fileInput = open(folders['small'] + fileName)
fileOutput = open(folders['small'] + fileName + '.json', 'w')

fileOutput.write('[')

lastCharacter = ','
posOutput = 0
for line in fileInput.readlines():
    if line.startswith('#'):
        continue

    if posOutput == 255:
        lastCharacter = ']'

    line = line.split()
    if line[1] == electrode:
        fileOutput.write(line[3] + lastCharacter)
        posOutput += 1

fileOutput.close()


#plotar pelo matplotlib separado para cada eletrodo